# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


In [3]:
file_path_list[:3]

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv']

In [4]:
!head '/home/workspace/event_data/2018-11-27-events.csv'

artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userId
Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.54051E+12,961,Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile),200,1.54328E+12,88
Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.54051E+12,961,Woodstock Inprovisation,200,1.54328E+12,88
Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.54051E+12,961,Majesty (LP Version),200,1.54328E+12,88
The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.54087E+12,107,Love Shack,200,1.54328E+12,38
Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-N

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [7]:
import pandas as pd
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [8]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [9]:
# Create a Keyspace 
CREATE_KEYSPACE_QUERY="""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH replication = 
{'class':'SimpleStrategy', 'replication_factor' : 1}
"""

try:
    session.execute(CREATE_KEYSPACE_QUERY)
except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [11]:
# Double checking with pandas what I want
df.query('sessionId==338 and itemInSession==4')[['artist', 'song', 'length']]

,artist,song,length
444,Faithless,Music Matters (Mark Knight Dub),495.3073


In [12]:
df[['artist', 'song', 'length', 'sessionId', 'itemInSession']].head(2)

,artist,song,length,sessionId,itemInSession
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,277.15873,961,0
1,Jimi Hendrix,Woodstock Inprovisation,239.82975,961,1


In [40]:
#query = "drop table songplay_from_session"
#rows = session.execute(query)
#print(rows)

In [41]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS songplay_from_session"
query = query + """
(sessionId int, 
itemInSession int, 
artist text, 
song text, 
length float, 
PRIMARY KEY (sessionId, itemInSession))"""

try:
    session.execute(query)
except Exception as e:
    print(e) 

In [42]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


In [44]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

## Assign the INSERT statements into the `query` variable
QUERY_INSERT = "INSERT INTO songplay_from_session (sessionId, itemInSession, artist, song, length)"
QUERY_INSERT = QUERY_INSERT + " VALUES (%s, %s, %s, %s, %s)"
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        values_insert = (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5]))
        session.execute(QUERY_INSERT, values_insert)
        #print(values_insert)

#### Do a SELECT to verify that the data have been inserted into each table

In [45]:
# Double checking with pandas what I want
df.query('sessionId==338 and itemInSession==4')[['artist', 'song', 'length']]

,artist,song,length
444,Faithless,Music Matters (Mark Knight Dub),495.3073


In [48]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
SELECT_QUERY = """SELECT artist, song, length
FROM songplay_from_session
WHERE sessionId=338
  AND itemInSession=4"""

try:
    rows = session.execute(SELECT_QUERY)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [57]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


In [58]:
df.query('userId==10 and sessionId==182').sort_values(by='itemInSession')[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
4705,Three Drives,Greece 2000,Sylvie,Cruz
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [75]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_song_by_session"
query = query + """
(userId int, 
sessionId int,
itemInSession int, 
artist text, 
song text, 
firstName text,
lastName text,
PRIMARY KEY ((userId, sessionId), itemInSession))""" #composite partition key

try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [76]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88


In [77]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

## Assign the INSERT statements into the `query` variable
QUERY_INSERT = """INSERT INTO user_song_by_session (
userId, 
sessionId, 
iteminSession, 
artist, 
song, 
firstName, 
lastName)"""
QUERY_INSERT = QUERY_INSERT + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        values_insert = (int(line[10]),
                         int(line[8]),
                         int(line[3]), 
                         str(line[0]), 
                         str(line[9]), 
                         str(line[1]), 
                         str(line[4]),
                        )
        session.execute(QUERY_INSERT, values_insert)
        #print(values_insert)

In [49]:
df.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88


In [56]:
df.query('userId==10 and sessionId==182').sort_values(by='itemInSession')[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
4705,Three Drives,Greece 2000,Sylvie,Cruz
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [78]:
## name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
SELECT_QUERY = """SELECT artist, song, firstName, lastName
FROM user_song_by_session
WHERE userId=10
  AND sessionId=182
ORDER BY itemInSession"""

try:
    rows = session.execute(SELECT_QUERY)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'                

In [81]:
df.query("song=='All Hands Against His Own'")[['firstName', 'lastName']]

,firstName,lastName
2792,Tegan,Levine
5135,Sara,Johnson
6298,Jacqueline,Lynch


In [82]:
query = "CREATE TABLE IF NOT EXISTS user_by_song"
query = query + """
(userId int, 
song text, 
firstName text,
lastName text,
PRIMARY KEY ((song), userId))"""

try:
    session.execute(query)
except Exception as e:
    print(e) 
                    

In [83]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

## Assign the INSERT statements into the `query` variable
QUERY_INSERT = """INSERT INTO user_by_song (
userId,  
song, 
firstName, 
lastName)"""
QUERY_INSERT = QUERY_INSERT + " VALUES (%s, %s, %s, %s)"
    
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        values_insert = (int(line[10]),
                         str(line[9]), 
                         str(line[1]), 
                         str(line[4]),
                        )
        session.execute(QUERY_INSERT, values_insert)
        #print(values_insert)

In [84]:
# Give me every user name (first and last) in my music app history 
# who listened to the song 'All Hands Against His Own'
SELECT_QUERY = """SELECT firstName, lastName
FROM user_by_song
WHERE song='All Hands Against His Own'"""

try:
    rows = session.execute(SELECT_QUERY)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [85]:
## TO-DO: Drop the table before closing out the sessions

query = "DROP TABLE songplay_from_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


query = "DROP TABLE user_song_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE user_by_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [86]:
session.shutdown()
cluster.shutdown()